In [46]:
import pandas as pd
import numpy as np

In [3]:
# tried office365 API to access via url and didn't work due to issues with the installation of package
# download the file and save it in the same path
#url = 'https://lanecouncilofgovernments-my.sharepoint.com/:x:/g/personal/sshoaf_lcog_org/EaiRosQicVhNrRvpKjgrZ9cByrC1Z6D7esomrXNvMsHxUw?e=VFaLaP'
xl = pd.ExcelFile('Working DRAFT 2045 Project List.xlsx')
xl.sheet_names

['Auto Constrained - Arterial Lin',
 'Auto Constrained - Added Fwy',
 'Auto Constrained - Arterial Cap',
 'Auto Constrained - New Collecto',
 'Auto Constrained - Urban Stnds',
 'Auto Constrained - Study',
 'Auto Constrained - TOD',
 'Auto Illustrative - New Arteria',
 'Auto Illustrative - New Collect',
 'Auto Illustrative - Art Cap',
 'Auto Illustrative - Added Freew',
 'Auto Illustrative - Urban Stnds',
 'Transit Constrained',
 'Transit Illustrative',
 'Bike Constrained - woutRd',
 'Bike Constrained - wRd',
 'Bike Constrained - onstreet w',
 'Bike Constrained - onstreet wou',
 'Bike Illustrative - woutRD',
 'Bike Illustrative - withRd',
 'Bike Illustrative - onstreet w',
 'Table Data',
 'Bike Illustrative onstreet wout']

In [4]:
sheet_name = xl.sheet_names[0]

In [142]:
df = xl.parse('Auto Illustrative - New Arteria')

In [144]:
df.shape

(0, 14)

In [49]:
df = df.drop(labels=0, axis=0)

In [12]:
list(df.columns)

['Name',
 'Geographic Limits',
 'Description',
 'Primary\nJurisdiction',
 'Air Quality Status',
 'Estimated Cost (20XX)',
 'Estimated Year of Construction\n(4-Year Window)',
 'Year of Construction\nCost Range',
 'Unnamed: 8',
 'Length',
 'RTP #',
 'Federal Functional Class',
 'Jurisdictional Project #',
 'Project Status? (Completed, incomplete, etc.)']

In [13]:
import re

In [16]:
sheetList = xl.sheet_names
 
pattern = "Auto Constrained"
 
[sheet for sheet in sheetList if re.match(pattern, sheet)]

['Auto Constrained - Arterial Lin',
 'Auto Constrained - Added Fwy',
 'Auto Constrained - Arterial Cap',
 'Auto Constrained - New Collecto',
 'Auto Constrained - Urban Stnds',
 'Auto Constrained - Study',
 'Auto Constrained - TOD']

In [50]:
df['RTP #'].unique()

array([512, 27, 30], dtype=object)

In [18]:
def getSheetnames(excel='Working DRAFT 2045 Project List.xlsx',
                 pattern='Auto Constrained'):
    xl = pd.ExcelFile(excel)
    sheetList = xl.sheet_names
    return [sheet for sheet in sheetList if re.match(pattern, sheet)]

In [118]:
def getRTPid(excel='Working DRAFT 2045 Project List.xlsx',
             sheet_name='Auto Constrained - Arterial Lin'):
    xl = pd.ExcelFile(excel)
    df = xl.parse(sheet_name)
    name = df['Name'][0].split(":")[1].lstrip()
    df = df.drop(labels=0, axis=0)
    rtplist = df['RTP #'].unique()
    strlist = [item for item in rtplist if type(item) == str]
    if len(strlist) > 0:
        for s in strlist:
            rtplist = rtplist[rtplist != s]
    b = [x for x in rtplist if str(x) != 'nan']
    res = [ele for ele in b if ele > 0]
    return name, res

In [91]:
sheet_name='Auto Constrained - Study'

In [92]:
df = xl.parse(sheet_name)

In [93]:
df['Name'][0].split(":")[1].lstrip()

'Study'

In [94]:
df = df.drop(labels=0, axis=0)

In [95]:
rtplist = df['RTP #'].unique()

In [96]:
rtplist

array([96, 835, 838, 608, 718, 823, 818, 849, 819, 916, 815, 824, 829,
       831, 917, 918, 26, 31, '\xa0', 210, 102, 557, '485, 454', 211, 555,
       nan], dtype=object)

In [124]:
[item for item in df['RTP #'].unique() if type(item) is str]

['\xa0', '485, 454']

In [125]:
rtplist = df['RTP #'].unique()
strlist = [item for item in rtplist if type(item) is str]
if len(strlist) > 0:
    for s in strlist:
        rtplist = rtplist[rtplist != s]

In [136]:
a = rtplist
a

array([96, 835, 838, 608, 718, 823, 818, 849, 819, 916, 815, 824, 829,
       831, 917, 918, 26, 31, 210, 102, 557, 211, 555, nan], dtype=object)

In [141]:
a = a.astype(float)
a[~np.isnan(a)]

array([ 96., 835., 838., 608., 718., 823., 818., 849., 819., 916., 815.,
       824., 829., 831., 917., 918.,  26.,  31., 210., 102., 557., 211.,
       555.])

In [146]:
# check the project list in 2040
xl = pd.ExcelFile('2040 Project List_Consolidated draft with AQ (ORIGINAL).xlsx')
xl.sheet_names

['Auto Constrained - Arterial Lin',
 'Auto Constrained - Added Freewa',
 'Auto Constrained - Arterial Cap',
 'Auto Constrained - New Collecto',
 'Auto Constrained - Urban Stnds',
 'Auto Constrained - Study',
 'Auto Constrained - TOD',
 'Auto Illustrative - New Arteria',
 'Auto Illustrative - New Collect',
 'Auto Illustrative - Art Cap',
 'Auto Illustrative - Added Freew',
 'Auto Illustrative - Urban Stds',
 'Transit Constrained',
 'Transit Illustrative',
 'Bike Constrained - woutRd',
 'Bike Constrained - wRd',
 'Bike Constrained - onstreet w',
 'Bike Constrained - onstreet wou',
 'Bike Illustrative - woutRD',
 'Bike Illustrative - withRd',
 'Bike Illustrative - onstreet w',
 'Bike Illustrative onstreet wout',
 'Table Data']